In [48]:
import os
import pandas as pd
import numpy as np
import joblib
import importlib
from pathlib import Path
from sqlalchemy.orm import sessionmaker
import matplotlib.pyplot as plt
from pprint import pprint
import seaborn as sns

from sklearn import metrics
from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM
from keras.regularizers import l1, l2, l1_l2
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier as scikit_KerasClassifier
from scikeras.wrappers import KerasClassifier as scikeras_KerasClassifier
from imblearn.under_sampling import RandomUnderSampler

import sqlite_io as sio
import add_indicators as indic
import split_merge as sm
import balance
import model_mngr as modmgr

importlib.reload(sio)

<module 'sqlite_io' from 'C:\\Projets\\MarketDataEnrichment\\dataset_mngr\\sqlite_io.py'>

In [2]:
PATH_DATA = "C:\\Projets\\Data"
PATH_DB_FWK="C:\\Projets\\Data\\sqlite\\dataset_market.db"
PATH_DB_STOCK="C:\\Projets\\Data\\sqlite\\dataset_paris_stock_adjusted.db"
PATH_DATA_DTS=PATH_DATA+"\\DTS_FULL\\"

SUFFIX_TRAIN="_TRAIN.zip"
SUFFIX_VAL="_VAL.zip"
SUFFIX_CONF="_CONF.zip"

CONNECTION TO SQLITE

In [3]:
if "con_stock" in locals():
        sio.close_connection(con_stock)
con_stock = sio.get_connection(str_db_path=PATH_DB_STOCK)

if "con_fwk" in locals():
        sio.close_connection(con_fwk)
con_fwk = sio.get_connection(str_db_path=PATH_DB_FWK)

my_session_maker = sessionmaker(bind=con_fwk)
session=my_session_maker()

table_stock="DS_PARIS_1D_ADJ_CLEAN"

GET DATA

In [6]:
dts_name="PARIS_TREND_1D_20D_V2"
multi_symbol="PARIS_STOCK"

df_base=sio.get_candles_to_df(session=session,con=con_stock, target_table=table_stock,tradable=True)
df_work=pd.DataFrame()
for code_value in df_base.index.get_level_values('CODE').unique():
    sub_df=df_base[df_base.index.get_level_values('CODE') == code_value]
    df_work_tmp = indic.add_indicators_to_df(con=con_fwk, df_in=sub_df, dts_name=dts_name,symbol=multi_symbol)
    df_work = pd.concat([df_work, df_work_tmp])
    
df_work.sort_index(inplace=True)
df_work.info() 

DEBUG: SELECT CODE,OPEN_DATETIME, OPEN,HIGH,LOW,CLOSE,VOLUME FROM DS_PARIS_1D_ADJ_CLEAN can 
    WHERE can.TIMEFRAME=1440 AND can.SK_SYMBOL IN (SELECT SK_SYMBOL FROM SYMBOL WHERE TRADABLE=1)      


<string>:1: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
<string>:1: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
<string>:1: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
<string>:1: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to 

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1409355 entries, ('AB.PA', '2010-04-26') to ('XIL.PA', '2024-01-15')
Data columns (total 68 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   OPEN               1403182 non-null  float64
 1   HIGH               1403182 non-null  float64
 2   LOW                1403182 non-null  float64
 3   CLOSE              1403182 non-null  float64
 4   VOLUME             1403182 non-null  float64
 5   sma20              1397520 non-null  float64
 6   pos_sma20          1397520 non-null  float64
 7   sma50              1403182 non-null  float64
 8   sma200             1343880 non-null  float64
 9   pos_sma50          1403182 non-null  float64
 10  pos_sma200         1343880 non-null  float64
 11  pos_sma50_200      1343880 non-null  float64
 12  pos_sma20_50       1397520 non-null  float64
 13  rsi14              1405468 non-null  float64
 14  sma5_rsi14         1404272 non-null  float6

In [8]:
# df_work[10000:10010]
# pd.set_option('display.max_columns', None)
# print(df_work.describe())

df_work.round(5).to_csv(
    PATH_DATA_DTS+dts_name+"_BASE.zip", sep=",")

In [4]:
dts_name="PARIS_TREND_1D_20D_V2"
# dts_name="PARIS_TREND_1D_50D_V1"
multi_symbol="PARIS_STOCK"

df_work=pd.read_csv(PATH_DATA_DTS+dts_name+"_BASE.zip",sep=",",index_col=["OPEN_DATETIME","CODE"],parse_dates=["OPEN_DATETIME"])
df_work.head(5)

,,OPEN,HIGH,LOW,CLOSE,VOLUME,sma20,pos_sma20,sma50,sma200,pos_sma50,...,adx14,adx14_neg,adx14_pos,adx14_dif,avg_vol14,pos_avg_vol14,pos_sma20_200,williamsr_14,perf_sma_50_5d,perf_sma_200_5d
OPEN_DATETIME,CODE,,,,,,,,,,,,,,,,,,,,,
2010-04-26,AB.PA,12.98,12.98,12.20,12.68,62866.0,NaN,NaN,12.68000,NaN,0.00000,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-27,AB.PA,12.74,12.83,12.61,12.70,22370.0,NaN,NaN,12.69000,NaN,0.00079,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-28,AB.PA,12.70,12.70,12.41,12.50,8211.0,NaN,NaN,12.62667,NaN,-0.01003,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-29,AB.PA,12.60,12.65,12.46,12.64,4676.0,NaN,NaN,12.63000,NaN,0.00079,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-30,AB.PA,12.63,12.71,12.55,12.65,4470.0,NaN,NaN,12.63400,NaN,0.00127,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


Check Dataframe

In [5]:
# drop rows with no pos_sma200 
df_work=df_work.dropna(subset=['pos_sma200'])

# if williamsr_14 >0 =0 if williamsr_14<-100 = -100
# df_work['williamsr_14']=df_work['williamsr_14'].apply(lambda x: 0 if x>0 else x)
# df_work['williamsr_14']=df_work['williamsr_14'].apply(lambda x: -100 if x<-100 else x)

# if williamsr_14 >0 =0 if williamsr_14<-100 = -100
df_work.loc[df_work['williamsr_14'] > 0, 'williamsr_14'] = 0
df_work.loc[df_work['williamsr_14'] < -100, 'williamsr_14'] = -100

# print min and max of the columns williamsr_14, perf_sma_50_5d, perf_sma_200_5d
# print(f"{df_work['williamsr_14'].min()=}")  inf-100
# print(f"{df_work['williamsr_14'].max()=}") sup 0

# df_check=df_work[df_work['perf_sma_50_5d'] > 1]
# df_check=df_check[df_check['ret_1d'] <= 2]
# print(df_check.index.get_level_values('CODE').unique())
# df_check[df_check.index.get_level_values('CODE')=='AI.PA']
# df_check.head(5)
# df_check=df_work[df_work.index.get_level_values('CODE')=='AI.PA']
# CATG
# mask = df_work['stdev20_1d'] > 1000
# df_work.drop(df_work[mask].index, inplace=True)
# df_check[6000:6010]


In [6]:
df_work[10000:10010]

,,OPEN,HIGH,LOW,CLOSE,VOLUME,sma20,pos_sma20,sma50,sma200,pos_sma50,...,adx14,adx14_neg,adx14_pos,adx14_dif,avg_vol14,pos_avg_vol14,pos_sma20_200,williamsr_14,perf_sma_50_5d,perf_sma_200_5d
OPEN_DATETIME,CODE,,,,,,,,,,,,,,,,,,,,,
2020-06-02,ABEO.PA,7.6136,7.9789,7.6136,7.9789,6332.0,7.31078,0.09139,8.27304,17.04669,-0.03555,...,26.15567,22.77986,24.12433,1.34447,6752.40,0.93774,-0.57113,-31.81630,-0.03229,-0.02455
2020-06-03,ABEO.PA,8.1711,8.6710,8.1519,8.4595,8982.0,7.39729,0.14359,8.21825,16.96353,0.02936,...,25.89215,20.36515,32.16729,11.80214,5972.80,1.50382,-0.56393,-11.45907,-0.03301,-0.02453
2020-06-04,ABEO.PA,8.5557,8.5557,7.7866,8.4019,7735.0,7.48285,0.12282,8.18441,16.87961,0.02657,...,24.74737,23.41954,28.54604,5.12651,5537.15,1.39693,-0.55669,-16.46677,-0.03314,-0.02459
2020-06-05,ABEO.PA,8.4211,8.7864,8.2480,8.5749,6423.0,7.57514,0.13198,8.15596,16.79463,0.05137,...,24.10544,21.58743,29.66498,8.07755,5570.60,1.15302,-0.54895,-12.08848,-0.03011,-0.02463
2020-06-08,ABEO.PA,8.7672,9.0363,8.5941,8.7479,19545.0,7.65877,0.14221,8.12712,16.71052,0.07638,...,23.93590,20.13379,31.31449,11.18069,6324.85,3.09019,-0.54168,-14.42361,-0.02444,-0.02464
2020-06-09,ABEO.PA,8.7479,9.0363,8.3826,8.8056,8807.0,7.73568,0.13831,8.10039,16.62717,0.08706,...,23.25077,21.27235,28.39698,7.12462,6409.80,1.37399,-0.53476,-11.53788,-0.02087,-0.02461
2020-06-10,ABEO.PA,9.0363,9.0363,8.2673,8.6518,6484.0,7.81738,0.10674,8.09462,16.54402,0.06883,...,22.33359,20.60949,25.39907,4.78959,6102.05,1.06259,-0.52748,-19.22981,-0.01504,-0.02473
2020-06-11,ABEO.PA,8.5557,8.5557,8.0173,8.1711,5301.0,7.88372,0.03645,8.07194,16.45990,0.01229,...,20.89547,22.22109,23.22080,0.99971,6320.40,0.83871,-0.52104,-50.00289,-0.01374,-0.02486
2020-06-12,ABEO.PA,7.9981,8.0750,7.5175,7.6328,11060.0,7.91351,-0.03547,8.04233,16.37550,-0.05092,...,20.25323,26.93381,21.20351,-5.73030,6756.90,1.63685,-0.51675,-81.11310,-0.01393,-0.02496


In [7]:
df_work = indic.drop_indicators_by_type(
    con=con_fwk, df_in=df_work, dts_name=dts_name, symbol=multi_symbol, ind_type=0)
list_label = indic.get_ind_list_by_type_for_dts(
    con=con_fwk, dts_name=dts_name, symbol_code=multi_symbol, ind_type=2)
print(list_label)

           LABEL
0   lab_perf_20d
1   lab_perf_50d
2  lab_perf_125d


In [8]:
df_work=df_work.droplevel('CODE')
df_work.sort_index(inplace=True)
df_work.head(5)

,OPEN,HIGH,LOW,CLOSE,VOLUME,pos_sma20,pos_sma50,pos_sma200,pos_sma50_200,pos_sma20_50,...,pos_donchian20_lo,adx14,adx14_neg,adx14_pos,adx14_dif,pos_avg_vol14,pos_sma20_200,williamsr_14,perf_sma_50_5d,perf_sma_200_5d
OPEN_DATETIME,,,,,,,,,,,,,,,,,,,,,
1989-10-20,2.1507,2.1913,2.1406,2.1507,209645.0,-0.04857,-0.07186,0.02924,0.10893,-0.02448,...,0.00853,21.40482,44.89673,17.73996,-27.15677,0.27907,0.08178,-92.52260,-0.01153,NaN
1989-10-23,2.1467,2.1649,2.1325,2.1467,1342573.0,-0.04748,-0.07069,0.02695,0.10507,-0.02437,...,0.00666,23.06701,43.77485,16.73968,-27.03516,1.68277,0.07814,-94.16598,-0.01256,NaN
1989-10-24,2.0919,2.1568,2.0716,2.0919,1785758.0,-0.06722,-0.09114,0.00051,0.10084,-0.02565,...,0.00980,25.23754,47.58947,14.43476,-33.15471,2.13044,0.07260,-93.32895,-0.01421,NaN
1989-10-25,2.0817,2.0959,2.0716,2.0817,1091222.0,-0.06742,-0.09234,-0.00458,0.09669,-0.02672,...,0.00488,27.25303,45.65847,13.84906,-31.80942,1.27752,0.06739,-96.68091,-0.01586,NaN
1989-10-26,2.0899,2.1101,2.0899,2.0899,581543.0,-0.06020,-0.08556,-0.00100,0.09247,-0.02699,...,0.00883,28.67357,43.43999,15.60559,-27.83440,0.70232,0.06299,-93.98620,-0.01674,NaN


In [16]:
lab_studied = "lab_perf_20d"
algo_studied = "LSTM_CLASS"
dts_name="PARIS_TREND_1D_20D_V2"

df_work_lab = indic.drop_indicators_not_selected(con=con_fwk, df_in=df_work, dts_name=dts_name, symbol=multi_symbol,label=lab_studied,algo=algo_studied)
# print(df_work_lab.head(5))

df_split=sm.split_df_by_label_strat(
    df_in=df_work_lab, list_label=[lab_studied], split_timeframe="M",random_split=False,split_strat=(80,10,10))
df_selected = df_split['df_'+lab_studied+'_train']
df_valid = df_split['df_'+lab_studied+'_valid']
df_confirm = df_split['df_'+lab_studied+'_confirm']
df_selected.sort_index(inplace=True)
df_valid.sort_index(inplace=True)
df_confirm.sort_index(inplace=True)

print(f"selected: {df_selected.shape=} valid: {df_valid.shape=} confirm: {df_confirm.shape=}")


selected: df_selected.shape=(844051, 28) valid: df_valid.shape=(233081, 28) confirm: df_confirm.shape=(247146, 28)


In [10]:
label=lab_studied
df_class=balance.add_class_by_lab_nb_lines(df_in=df_selected,str_label=lab_studied,nb_class=4,bool_replace_label=False)
min_max_lab_by_class = df_class.groupby(label+'_class')[label].agg(['min', 'max'])
print(min_max_lab_by_class)

                        min      max
lab_perf_20d_class                  
0                  -0.80165 -0.03860
1                  -0.03859  0.00610
2                   0.00611  0.05654
3                   0.05655  3.82176


In [17]:
label=lab_studied
df_class=balance.add_class_by_lab_nb_lines(df_in=df_selected,str_label=lab_studied,nb_class=4,bool_replace_label=True)
df_class.sort_index(inplace=True)
# categ_50={0:[-1,-0.05456],1:[-0.07876,-0.00783],2:[-0.00783,0.04790],3:[0.04790,0.12406],4:[0.12406,6]}
categ_20={0:[-1,-0.0386],1:[-0.0386,0.0061],2:[0.0061,0.05654],3:[0.05654,5]}
df_class_val=balance.add_lab_by_class(df_in=df_valid,str_label=lab_studied, categ=categ_20,bool_replace_label=True) # categ
df_class_val.sort_index(inplace=True)
df_class_conf=balance.add_lab_by_class(df_in=df_confirm,str_label=lab_studied, categ=categ_20,bool_replace_label=True) # categ
df_class_conf.sort_index(inplace=True)
print(df_class.loc[:, label].dropna().iloc[[0, -1]])
print(df_class_val.loc[:, label].dropna().iloc[[0, -1]])
print(df_class_conf.loc[:, label].dropna().iloc[[0, -1]])
# df_class_clean=df_class.drop(['OPEN','HIGH','LOW','CLOSE','VOLUME','lab_perf_125d','lab_perf_20d','lab_perf_50d'],axis=1)
data = df_class[label]
print(data.value_counts().sort_index())
data_val = df_class_val[label]
print(data_val.value_counts().sort_index())
data_conf = df_class_conf[label]
print(data_conf.value_counts().sort_index())
# min_max_lab_by_class = df_class.groupby(label+'_class')[label].agg(['min', 'max'])
# print(min_max_lab_by_class)

# lab_perf_20d : train min nb rows 211000 validation 53000 confirm 55000

OPEN_DATETIME
1989-10-27    3
2017-02-28    2
Name: lab_perf_20d, dtype: int64
OPEN_DATETIME
2017-03-01    2.0
2020-07-31    2.0
Name: lab_perf_20d, dtype: float64
OPEN_DATETIME
2020-08-03    2.0
2023-12-13    1.0
Name: lab_perf_20d, dtype: float64
lab_perf_20d
0    211018
1    211018
2    211014
3    211001
Name: count, dtype: int64
lab_perf_20d
0.0    64761
1.0    58466
2.0    56807
3.0    53038
Name: count, dtype: int64
lab_perf_20d
0.0    73920
1.0    56794
2.0    55138
3.0    61292
Name: count, dtype: int64


In [18]:
list_feat = df_class.columns.values.tolist()
list_feat.remove(label)
df_x_train, col_y_train = sm.split_df_x_y(
    df_in=df_class, list_features=list_feat, str_label=label, drop_na=True)
df_x_val, col_y_val = sm.split_df_x_y(
    df_in=df_class_val, list_features=list_feat, str_label=label, drop_na=True)
df_x_conf, col_y_conf = sm.split_df_x_y(
    df_in=df_class_conf, list_features=list_feat, str_label=label, drop_na=True)

df_class.round(5).to_csv(
    PATH_DATA_DTS+dts_name+SUFFIX_TRAIN, sep=",")
df_class_val.round(5).to_csv(
    PATH_DATA_DTS+dts_name+SUFFIX_VAL, sep=",")
df_class_conf.round(5).to_csv(
    PATH_DATA_DTS+dts_name+SUFFIX_CONF, sep=",")

Calculate and save scaler

In [5]:
dts_name="PARIS_TREND_1D_20D_V2"
multi_symbol="PARIS_STOCK"
label = "lab_perf_20d"

df_class=pd.read_csv(PATH_DATA_DTS+dts_name+SUFFIX_TRAIN,sep=",",index_col=["OPEN_DATETIME"],parse_dates=["OPEN_DATETIME"])
df_class=df_class.dropna(subset=[label])
df_class=df_class.sort_index()

df_norm,norm_scaler= balance.normalize_df(df_in=df_class,str_label=label,tuple_ft_range=(-1,1))

file_name=dts_name+"_train_colab_lstm_norm_2405"
scaler_name=file_name+"_scaler.save"
joblib.dump(norm_scaler,filename=PATH_DATA_DTS+scaler_name)

# df_class_val=pd.read_csv(PATH_DATA_DTS+dts_name+SUFFIX_VAL,sep=",",index_col=["OPEN_DATETIME"],parse_dates=["OPEN_DATETIME"])
# df_class_val.dropna(subset=[label], inplace=True)
# df_class_val.sort_index(inplace=True)

# list_feat = df_class.columns.values.tolist()
# list_feat.remove(label)
# X, y = sm.split_df_x_y(
#     df_in=df_class, list_features=list_feat, str_label=label, drop_na=True)
# nb_val=211000
# method = RandomUnderSampler(sampling_strategy={0:nb_val,1:nb_val,2:nb_val,3:nb_val}) 
# df_x_train, col_y_train=  method.fit_resample(X, y)
# print(col_y_train.value_counts().sort_index())

# X, y = sm.split_df_x_y(
#     df_in=df_class_val, list_features=list_feat, str_label=label, drop_na=True)
# nb_val=53000
# method = RandomUnderSampler(sampling_strategy={0:nb_val,1:nb_val,2:nb_val,3:nb_val}) # 53000 pour lab 20 et nn pour lab 50
# df_x_val, col_y_val=  method.fit_resample(X, y)
# print(col_y_val.value_counts().sort_index())

['C:\\Projets\\Data\\DTS_FULL\\PARIS_TREND_1D_20D_V2_train_colab_lstm_norm_2405_scaler.save']

Load train et val df, normalize,  undersample  and preparation for LSTM

In [50]:
dts_name="PARIS_TREND_1D_20D_V2"
multi_symbol="PARIS_STOCK"
label = "lab_perf_20d"
file_name=dts_name+"_train_colab_lstm_norm_2405"
scaler_name=file_name+"_scaler.save"
scaler=joblib.load(PATH_DATA_DTS+scaler_name)


df_class=pd.read_csv(PATH_DATA_DTS+dts_name+SUFFIX_TRAIN,sep=",",index_col=["OPEN_DATETIME"],parse_dates=["OPEN_DATETIME"])
df_class=df_class.dropna(subset=[label])
df_class=df_class.loc['1995-01-01':] # drop rows < 1995-01-01
df_class=df_class.sort_index()
df_class_val=pd.read_csv(PATH_DATA_DTS+dts_name+SUFFIX_VAL,sep=",",index_col=["OPEN_DATETIME"],parse_dates=["OPEN_DATETIME"])
df_class_val=df_class_val.dropna(subset=[label])
df_class_val=df_class_val.sort_index()

# normalize df_class and df_class_val
df_class_train_norm=balance.normalize_df_scaler(df_in=df_class, str_label=label,scaler=scaler)
df_class_val_norm=balance.normalize_df_scaler(df_in=df_class_val, str_label=label,scaler=scaler)

list_feat = df_class.columns.values.tolist()
list_feat.remove(label)
X, y = sm.split_df_x_y(
    df_in=df_class_train_norm, list_features=list_feat, str_label=label, drop_na=True)
nb_val=20000#211000
method = RandomUnderSampler(sampling_strategy={0:nb_val,1:nb_val,2:nb_val,3:nb_val}) 
df_x_train, col_y_train=  method.fit_resample(X, y)
# print(col_y_train.value_counts().sort_index())

X, y = sm.split_df_x_y(
    df_in=df_class_val_norm, list_features=list_feat, str_label=label, drop_na=True)
nb_val=5000#53000
method = RandomUnderSampler(sampling_strategy={0:nb_val,1:nb_val,2:nb_val,3:nb_val}) # 53000 pour lab 20 et nn pour lab 50
df_x_val, col_y_val=  method.fit_resample(X, y)
# print(col_y_val.value_counts().sort_index())

sequence_length = 10

x_train=df_x_train.values
y_train=col_y_train.values
x_val=df_x_val.values
y_val=col_y_val.values
x_train_lstm,y_train_lstm=sm.prepare_sequences(x_train,y_train,sequence_length)
x_val_lstm,y_val_lstm=sm.prepare_sequences(x_val,y_val,sequence_length)


Correlation

In [ ]:
corr_train = df_x_train.corr()
plt.clf()
fig, ax = plt.subplots(figsize=(6, 6))
corr_train.replace(1,0,inplace=True)
corr_train=corr_train.applymap(lambda x : None if x< 0.7 and x>-0.7 else x)
corr_train.dropna(axis=0,how='all',inplace=True)
corr_train.dropna(axis=1,how='all',inplace=True)

# corr_train_check=corr_train[corr_train >0.8]
corr_train_check=corr_train
sns.heatmap(corr_train_check, annot=False, cmap='coolwarm', vmin=-1, vmax=1, ax=ax)

In [ ]:
sns.scatterplot(data=df_class, x='pos_sma200', y='pos_top50', hue='lab_perf_20d', palette='Set1')

In [51]:
list_param_valid = [
                    {'fit__batch_size': 256, 'model__dropout': 0.05, 'model__layers': [64, 10], 'optimizer__lr': 0.1, 'optimizer__momentum': 0.9},
                    # {'fit__batch_size': 32, 'fit__epochs': 350, 'model__dropout': 0.05, 'model__layers': [128, 20], 'optimizer__lr': 0.1, 'optimizer__momentum': 0.7},
                    # {'fit__batch_size': 32, 'fit__epochs': 350, 'model__dropout': 0.05, 'model__layers': [128, 20], 'optimizer__lr': 0.1, 'optimizer__momentum': 0.5},
                    # {'fit__batch_size': 64, 'fit__epochs': 350, 'model__dropout': 0.05, 'model__layers': [128, 20], 'optimizer__lr': 0.1, 'optimizer__momentum': 0.9},
                    # {'fit__batch_size': 64, 'fit__epochs': 350, 'model__dropout': 0.05, 'model__layers': [128, 20], 'optimizer__lr': 0.1, 'optimizer__momentum': 0.7},
                    # {'fit__batch_size': 64, 'fit__epochs': 350, 'model__dropout': 0.05, 'model__layers': [128, 20], 'optimizer__lr': 0.1, 'optimizer__momentum': 0.5},
]

input_dim = x_train.shape[-1]
num_classes = 4
epochs = 20#350
suffix="lstm_v1"
filename_tmp_model = dts_name+"_"+suffix+".h5"
patience = 3

val_accuracy=0.0
obj_acc=0.4
cpt_param=0
try_limit=10
pct_check_class=0.75 # check if at least n% of the validation set per class

len_val=len(x_val_lstm)
check_class_limit=(len_val/num_classes)*pct_check_class
check_class=False # check if at least obj_acc accuracy per class

while(cpt_param<len(list_param_valid) and check_class==False):
    param_valid=list_param_valid[cpt_param]
    print(param_valid)
    cpt=0

    while(cpt<try_limit and check_class==False):
        cpt+=1
        es = EarlyStopping(monitor="val_loss", patience=patience, mode="min", verbose=2)
        mc = ModelCheckpoint(filename_tmp_model, monitor="val_loss",
                            mode="min", save_freq="epoch", save_best_only=True)
        lstm_model = scikeras_KerasClassifier(model=modmgr.create_scikeras_lstm_model, optimizer="adam",optimizer__momentum=param_valid['optimizer__momentum'],
                                            optimizer__lr=param_valid['optimizer__lr'], model__layers=param_valid['model__layers'], model__dropout=param_valid['model__dropout'],
                                                callbacks=[es, mc], verbose=1)

        history = lstm_model.fit(
            x_train_lstm, y_train_lstm, batch_size=param_valid['fit__batch_size'], epochs=epochs, validation_data=(x_val_lstm, y_val_lstm))

        train_loss = history.history_['loss']
        val_loss = history.history_['val_loss']

        # Plot loss
        # epochs_done = range(1, len(train_loss) + 1)
        # plt.plot(epochs_done, train_loss, 'bo-', label='Training Loss')
        # plt.plot(epochs_done, val_loss, 'ro-', label='Validation Loss')
        # plt.legend()
        # plt.show()

        saved_model = load_model(filename_tmp_model)
        # saved_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['loss'])
        # loss, accuracy = saved_model.evaluate(x_valid, y_valid)

        # Prediction on validation
        y_pred = saved_model.predict(x_val_lstm)
        y_pred_classes = np.argmax(y_pred, axis=1)

        # Accuracy on validation
        val_accuracy = metrics.accuracy_score(y_val_lstm.argmax(axis=1), y_pred_classes)
        print(f"Accuracy on Validation Set: {val_accuracy} {cpt=}")

        # check prediction au moins 30 par classe
        if val_accuracy>=obj_acc:
            check_class=True
            for i in range(num_classes):
                nb_lab=sum(y_pred_classes == i)
                if nb_lab<check_class_limit  :
                    check_class=False
                    print(f"Check class {i=} {nb_lab=} {check_class=} {check_class_limit=}")

    if cpt>=try_limit :
        cpt_param+=1
        print(f"Optim fail {cpt=} param suivant {cpt_param=}")

if cpt>=try_limit :
    print(f"Optim fail {cpt=}")

else :
    confusion = metrics.confusion_matrix(y_val_lstm.argmax(axis=1), y_pred_classes)
    print(confusion)

    for i in range(num_classes):
        print(f"Categ {i}: real {sum(y_val_lstm.argmax(axis=1) == i)} predict {sum(y_pred_classes == i)}")

    #check saved model
    saved_model = load_model(filename_tmp_model)
    y_pred = saved_model.predict(x_val_lstm)
    confusion = metrics.confusion_matrix(y_val_lstm.argmax(axis=1), y_pred.argmax(axis=1))
    print(confusion)

{'fit__batch_size': 256, 'fit__epochs': 50, 'model__dropout': 0.05, 'model__layers': [64, 10], 'optimizer__lr': 0.1, 'optimizer__momentum': 0.9}
Epoch 1/20
313/313 [==============================] - 38s 107ms/step - loss: 1.2458 - accuracy: 0.3762 - val_loss: 1.1770 - val_accuracy: 0.4029
Epoch 2/20
313/313 [==============================] - 38s 123ms/step - loss: 1.2078 - accuracy: 0.3997 - val_loss: 1.2138 - val_accuracy: 0.3849
Epoch 3/20
313/313 [==============================] - 35s 113ms/step - loss: 1.1952 - accuracy: 0.4099 - val_loss: 1.2075 - val_accuracy: 0.4029
Epoch 4/20
313/313 [==============================] - 39s 125ms/step - loss: 1.1903 - accuracy: 0.4145 - val_loss: 1.2040 - val_accuracy: 0.4087
Epoch 4: early stopping
625/625 [==============================] - 6s 8ms/step
Accuracy on Validation Set: 0.40293131909359214 cpt=1
[[2801  413  947  830]
 [ 527 1499 2409  565]
 [ 473 1385 2666  476]
 [2846  252  813 1089]]
Categ 0: real 4991 predict 6647
Categ 1: real 500

In [ ]:
input_dim = x_train.shape[-1]
window_size = sequence_length
dropout = 0.2
num_classes = 4

# cat_y_train = keras.utils.to_categorical(col_y_train, num_classes)
# cat_y_valid = keras.utils.to_categorical(col_y_valid, num_classes)

# df_x_train_exp = np.expand_dims(df_x_train, axis=2)
# df_x_valid_exp = np.expand_dims(df_x_valid, axis=2)


model_LSTM = Sequential()
model_LSTM.add(LSTM(units=20, return_sequences=False,#True
               input_shape=(window_size, input_dim)))
#,kernel_regularizer=l2(0.1), recurrent_regularizer=l2(0.1), bias_regularizer=l2(0.1)
model_LSTM.add(Dropout(rate=dropout))   
# model_LSTM.add(Dropout(rate=dropout))
# model_LSTM.add(Bidirectional(LSTM((window_size * 2), return_sequences=True)))
# model_LSTM.add(Dropout(rate=dropout))
# model_LSTM.add(Bidirectional(LSTM(window_size, return_sequences=False)))
model_LSTM.add(Dense(units=num_classes, activation='softmax'))

model_LSTM.compile(loss='categorical_crossentropy',
                   optimizer='adam', metrics=['accuracy'])

history = model_LSTM.fit(x_train_lstm, y_train_lstm, batch_size=1024,
                         shuffle=False, epochs=20, validation_data=(x_val_lstm, y_val_lstm))#,verbose=0

train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Plot loss
epochs = range(1, len(train_accuracy) + 1)
plt.plot(epochs, train_accuracy, 'bo-', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'ro-', label='Validation accuracy')
plt.legend()
plt.show()
